In [74]:
import pandas as pd
import os

data_folder = 'OfficerIDbyYear'
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# Load CSV (Employee data) into a dictionary of DataFrames (one for each year)
employee_data = {}
years = range(2010, 2022)

for csv_file in csv_files:
    # Check if the filename matches the pattern
    if 'Order of Councilor Arroyo' in csv_file:  
        # Extract the year from the filename by splitting based on space and dot
        year = csv_file.split(' - ')[-1].split('.')[2] 
        
        # Load the data for that year
        file_path = os.path.join(data_folder, csv_file)
        year_data = pd.read_csv(file_path)
        
        # remove officer if they are inactive
        year_data.columns = year_data.columns.str.strip().str.lower()
            
        employee_data[year] = year_data[year_data["reason inactive"].isnull()]

# print(employee_data['2012'].head())
print(employee_data.keys())

dict_keys(['2021', '2020', '2017', '2011', '2010', '2013', '2018', '2016', '2022', '2012', '2015', '2019', '2014'])


In [75]:
import string

# load incident data
incident_data = pd.read_csv("normalized_cases.csv")

# create year column 
incident_data['Incident Year'] = pd.to_datetime(incident_data['Inc: Received date']).dt.year
incident_data['Incident Year'] = incident_data['Incident Year'].dropna()
incident_data['Incident Year'] = incident_data['Incident Year'].astype(int)

# create a full name column
incident_data['Middle_Initial'] = incident_data['Middle_Initial'].astype(str)
incident_data['Middle_Initial'] = incident_data['Middle_Initial'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
incident_data['Full Name'] = incident_data['First_Name'] + " " + incident_data['Middle_Initial'] + " " + incident_data['Off: Last name']

incident_data.head()

,Unnamed: 0,Inc: IA No,Inc: Incident type,Inc: Received date,Inc: Occurred date,OffSnp: Title/rank,Off: First name,Off: Last name,Alg: Allegation,Alg: Finding,Act: Action taken,Act: Days/hours suspended,Act: Action taken date,First_Name,Middle_Initial,Incident Year,Full Name
0,0,IAD2011-0003,Citizen complaint,01/05/2011,06/18/2009,Sergeant Detective,Albert M,Terestre,Use of Force,Exonerated,NaN,0.0,NaN,Albert,M,2011,Albert M Terestre
1,1,IAD2011-0003,Citizen complaint,01/05/2011,06/18/2009,Police Officer,John K,Rogers,Use of Force,Exonerated,NaN,0.0,NaN,John,K,2011,John K Rogers
2,2,IAD2011-0004,Citizen complaint,01/05/2011,01/01/2011,Detective,Scott M,Mackie,Conformance to Laws,Unfounded,NaN,NaN,NaN,Scott,M,2011,Scott M Mackie
3,3,IAD2011-0011,Citizen complaint,01/07/2011,01/03/2011,Police Officer,Michael J.,Olszak,Neg.Duty/Unreasonable Judge,Exonerated,NaN,0.0,NaN,Michael,J,2011,Michael J Olszak
4,4,IAD2011-0011,Citizen complaint,01/07/2011,01/03/2011,Police Officer,Angel,Figueroa,Neg.Duty/Unreasonable Judge,Exonerated,NaN,0.0,NaN,Angel,nan,2011,Angel nan Figueroa


In [76]:
def get_employee_id(row):
    # Get the year from incident data
    year = str(row['Incident Year'])
    
    # Check if the year is present in the employee data for that year
    year_data = employee_data.get(year)
    if year_data is not None:
        # Create Full Name in Employee Data
        year_data['Full Name'] = year_data['first name'] + " " + year_data['middle'] + " " + year_data['last']
        
        # Find matching Full Name and return Employee ID
        matching_row = year_data[year_data['Full Name'] == row['Full Name']]
        
        if not matching_row.empty:
            return matching_row.iloc[0]['id']
        
    return None  

In [79]:
# Apply the function to the Incident Data to add the Employee ID
incident_data['Employee ID'] = incident_data.apply(get_employee_id, axis=1)
incident_data = incident_data.dropna(subset=['Employee ID'])
incident_data['Employee ID'] = incident_data['Employee ID'].astype('Int64')

# Save the result to a new CSV file
incident_data.to_csv("incident_with_employee_ids.csv", index=False)
print(incident_data.head())

    Unnamed: 0    Inc: IA No Inc: Incident type Inc: Received date  \
0            0  IAD2011-0003  Citizen complaint         01/05/2011   
1            1  IAD2011-0003  Citizen complaint         01/05/2011   
8            8  IAD2011-0013  Citizen complaint         01/07/2011   
16          16  IAD2011-0023  Citizen complaint         01/14/2011   
21          21  IAD2011-0039  Citizen complaint         01/22/2011   

   Inc: Occurred date  OffSnp: Title/rank Off: First name Off: Last name  \
0          06/18/2009  Sergeant Detective        Albert M       Terestre   
1          06/18/2009      Police Officer          John K         Rogers   
8          10/27/2010      Police Officer        Andrew B          Blake   
16         11/01/2012           Detective         Laura A        Delaney   
21         01/22/2011      Police Officer       Richard L         Medina   

                Alg: Allegation   Alg: Finding Act: Action taken  \
0                  Use of Force     Exonerated        

In [ ]:
# create new CSV to only keep employee ID and employee name
employee_id = incident_data.drop_duplicates(subset=['Employee ID'], keep='first')

employee_id.to_csv("unique_employee_ids.csv", index=False)

print(employee_id.head())

    Unnamed: 0    Inc: IA No Inc: Incident type Inc: Received date  \
0            0  IAD2011-0003  Citizen complaint         01/05/2011   
1            1  IAD2011-0003  Citizen complaint         01/05/2011   
8            8  IAD2011-0013  Citizen complaint         01/07/2011   
16          16  IAD2011-0023  Citizen complaint         01/14/2011   
21          21  IAD2011-0039  Citizen complaint         01/22/2011   

   Inc: Occurred date  OffSnp: Title/rank Off: First name Off: Last name  \
0          06/18/2009  Sergeant Detective        Albert M       Terestre   
1          06/18/2009      Police Officer          John K         Rogers   
8          10/27/2010      Police Officer        Andrew B          Blake   
16         11/01/2012           Detective         Laura A        Delaney   
21         01/22/2011      Police Officer       Richard L         Medina   

                Alg: Allegation   Alg: Finding Act: Action taken  \
0                  Use of Force     Exonerated        

In [4]:
import pandas as pd
import os

employee_id = pd.read_csv("employeeroster.csv")
employee_id.head()

,Unnamed: 0,Inc: IA No,Inc: Incident type,Inc: Received date,Inc: Occurred date,OffSnp: Title/rank,Off: First name,Off: Last name_incident,Alg: Allegation,Alg: Finding,...,retro_pay,other_pay,ot_pay,injured_pay,detail_pay,quinn_pay,total_pay,year,rank_bucket,total_pay_percentile
0,1,IAD2011-0003,Citizen complaint,01/05/2011,06/18/2009,Police Officer,John K,Rogers,Use of Force,Exonerated,...,0.00,7999.29,28703.19,21057.91,4345.0,0.00,113039.50,2013,Officer,28.884491
1,8,IAD2011-0013,Citizen complaint,01/07/2011,10/27/2010,Police Officer,Andrew B,Blake,Respectful Treatment,Not Sustained,...,0.00,800.00,25059.02,0.00,30370.0,3886.32,129501.14,2011,Officer,44.297860
2,16,IAD2011-0023,Citizen complaint,01/14/2011,11/01/2012,Detective,Laura A,Delaney,Neg.Duty/Unreasonable Judge,Sustained,...,0.00,5117.79,2383.58,56994.82,0.0,9275.04,90975.96,2012,Detective,12.833480
3,21,IAD2011-0039,Citizen complaint,01/22/2011,01/22/2011,Police Officer,Richard L,Medina,Self Identification,Exonerated,...,3011.25,850.00,47019.10,44.28,296.0,0.00,126523.13,2014,Sergeant,41.512753
4,22,IAD2011-0039,Citizen complaint,01/22/2011,01/22/2011,Police Officer,James H,LaCroix,Respectful Treatment,Not Sustained,...,NaN,3300.00,1277.54,69071.74,5336.0,NaN,96547.23,2016,Officer,16.274553


In [6]:
police_pay = pd.read_csv("bpi_police_financial.csv")
police_pay.head()

,bpi_id,employee_id,badge_no,last_name,first_name,rank,unit,union_code,race,sex,...,retro_pay,other_pay,ot_pay,injured_pay,detail_pay,quinn_pay,total_pay,year,rank_bucket,total_pay_percentile
0,691f1132-4760-44cb-8bf7-3bbdaefd0938,103384,5147.0,Gannon,Sean,Police Officer,District 06,BPP,WHITE,M,...,8.92,NaN,NaN,NaN,NaN,NaN,8.92,2017,Officer,0.000000
1,691f1132-4760-44cb-8bf7-3bbdaefd0938,103384,5147.0,Gannon,Sean,Police Officer,District 02,BPP,WHITE,M,...,8.92,NaN,NaN,NaN,NaN,NaN,8.92,2017,Officer,0.000000
2,02220257-5141-424b-89fc-269de3c3b6e2,144486,5235.0,Donlon,Christopher,Police Officer,District 02,BPP,WHITE,M,...,23.70,NaN,NaN,NaN,NaN,NaN,23.70,2017,Officer,0.007329
3,02220257-5141-424b-89fc-269de3c3b6e2,144486,5235.0,Donlon,Christopher,Police Officer,Student Officers,BPP,WHITE,M,...,23.70,NaN,NaN,NaN,NaN,NaN,23.70,2017,Officer,0.007329
4,02220257-5141-424b-89fc-269de3c3b6e2,144486,5235.0,Donlon,Christopher,Police Officer,District 11,BPP,WHITE,M,...,23.70,NaN,NaN,NaN,NaN,NaN,23.70,2017,Officer,0.007329
